In [1]:
import findspark
findspark.init("/home/rodolfo/spark-3.3.1-bin-hadoop3")
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("rec").getOrCreate()

23/01/02 11:53:07 WARN Utils: Your hostname, rodolfo-300E5M-300E5L resolves to a loopback address: 127.0.1.1; using 10.0.0.107 instead (on interface wlp3s0)
23/01/02 11:53:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/02 11:53:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/01/02 11:53:17 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/01/02 11:53:17 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [4]:
data = spark.read.csv("../../data/movielens_ratings.csv", header=True, inferSchema=True)

In [5]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [6]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [7]:
train, test = data.randomSplit([0.8, 0.2])

In [10]:
als = ALS(
    maxIter=5, 
    regParam=0.01, 
    userCol="userId", 
    itemCol="movieId", 
    ratingCol="rating"
).fit(train)

In [11]:
predictions = als.transform(test)

In [12]:
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|      1|   1.0|    28| 0.92057586|
|      1|   1.0|    26|  0.5703187|
|      2|   1.0|    26|-0.47901368|
|      2|   1.0|    12|    1.11681|
|      3|   1.0|     1|   1.900561|
|      0|   1.0|    13|  1.6155001|
|      3|   1.0|    13|  1.4834387|
|      6|   1.0|    13|  1.0417577|
|      2|   1.0|    16|   3.877215|
|      1|   1.0|     5| 0.28776595|
|      4|   1.0|     5|  2.8921201|
|      1|   1.0|    19|  0.8123348|
|      1|   4.0|    15| 0.71457213|
|      5|   1.0|     9| 0.36254102|
|      4|   2.0|     8|-0.08722085|
|      2|   1.0|    23|  2.7548623|
|      7|   1.0|     7| 0.43609408|
|      1|   3.0|    25| 0.29889256|
|      5|   1.0|    14|-0.36040413|
|      5|   2.0|     0|   2.875648|
+-------+------+------+-----------+
only showing top 20 rows



In [13]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")

In [14]:
rmse = evaluator.evaluate(predictions)

In [25]:
print(f"RMSE {rmse:.4f}")

RMSE 1.7081


In [26]:
single_user = test.filter(test["userId"]==11).select(["movieId", "userId"])

In [27]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|     19|    11|
|     23|    11|
|     27|    11|
|     30|    11|
|     40|    11|
|     59|    11|
|     77|    11|
|     82|    11|
|     97|    11|
+-------+------+



In [28]:
recommendations = als.transform(single_user)

In [29]:
recommendations.orderBy("prediction", ascending=False).show()

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|     27|    11| 5.6239734|
|     23|    11|  4.068678|
|     59|    11|   4.00388|
|     30|    11| 3.7098498|
|     19|    11| 2.4077039|
|     97|    11| 0.9831735|
|     82|    11|0.51282233|
|     77|    11|0.11847764|
|     40|    11|-1.8592246|
+-------+------+----------+

